<a href="https://colab.research.google.com/github/LCaravaggio/politext/blob/snowball/Parseo_de_OrdenesdelDia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import numpy as np


import json
from google.colab import drive

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

drive.mount('/content/drive', force_remount=True)
with open("/content/drive/My Drive/kaggle.json", 'r') as f:
    api_token= json.load(f)

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d leonardocaravaggio/scraps-pdfs -f "Scraps PDFs.csv"
!kaggle datasets download -d leonardocaravaggio/scraps-pdfs -f "base_formateada_senado_snowball.csv"

mkdir: cannot create directory ‘/root/.kaggle’: File exists
Mounted at /content/drive
Scraps%20PDFs.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
 96% 137M/143M [00:02<00:00, 97.0MB/s]
100% 143M/143M [00:02<00:00, 62.9MB/s]


In [12]:
import zipfile
import os

os.listdir()

for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

In [13]:
# Cargamos la base
base = pd.read_csv('/content/Scraps PDFs.csv')
base_parseada = pd.read_csv('/content/base_formateada_senado_snowball.csv')

<ipython-input-13-e0eeee1141a8>:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  base_parseada = pd.read_csv('/content/base_formateada_senado_snowball.csv')


In [14]:
base.link[10]

'https://www.senado.es/legis10/publicaciones/pdf/senado/ds/DS_P_10_11.PDF'

In [ ]:
def generar(l):
  df=pd.DataFrame()


  return df

In [ ]:
for row in base_parseada.values:
  base2 = pd.concat([base2, generar(row)], axis=0)

In [ ]:
base2=base2.reset_index(drop=True)

In [ ]:
base2['fecha_formato'] = pd.to_datetime(base2['fecha_formato'], errors='coerce')

In [ ]:
grupos=pd.read_xml('/content/openData2023731165138.xml')

In [ ]:
base2['nombre']=""
base2['apellidos']=""
base2['procedLiteral']=""
base2['procedLugar']=""
base2['grupoNombre']=""


lista = []

for linea in base2.values:

    nombres = grupos.loc[grupos['apellidos'] == linea[3].upper()].nombre.tolist()

    if next(iter(nombres or []), None)==None:
      nombres = grupos.loc[grupos['apellidos'] == linea[2].upper()].nombre.tolist()
      linea[9]=next(iter(nombres or []), None)

      apellidos = grupos.loc[grupos['apellidos'] == linea[2].upper()].apellidos.tolist()
      linea[10]=next(iter(apellidos or []), None)

      procedLiteral	 = grupos.loc[grupos['apellidos'] == linea[2].upper()].procedLiteral.tolist()
      linea[11]=next(iter(procedLiteral or []), None)

      procedLugar	 = grupos.loc[grupos['apellidos'] == linea[2].upper()].procedLugar.tolist()
      linea[12]=next(iter(procedLugar or []), None)

      grupoNombre	= grupos.loc[grupos['apellidos'] == linea[2].upper()].grupoNombre.tolist()
      linea[13]=next(iter(grupoNombre or []), None)

    else:

      linea[9]=next(iter(nombres or []), None)

      apellidos = grupos.loc[grupos['apellidos'] == linea[3].upper()].apellidos.tolist()
      linea[10]=next(iter(apellidos or []), None)

      procedLiteral	 = grupos.loc[grupos['apellidos'] == linea[3].upper()].procedLiteral.tolist()
      linea[11]=next(iter(procedLiteral or []), None)

      procedLugar	 = grupos.loc[grupos['apellidos'] == linea[3].upper()].procedLugar.tolist()
      linea[12]=next(iter(procedLugar or []), None)

      grupoNombre	= grupos.loc[grupos['apellidos'] == linea[3].upper()].grupoNombre.tolist()
      linea[13]=next(iter(grupoNombre or []), None)

    lista.append(linea)

In [ ]:
base2=pd.DataFrame(lista, columns=base2.columns)

In [ ]:
column_name = 'apellidos'
null_count = base2[column_name].isnull().sum()

In [ ]:
null_count

410091

In [ ]:
base2[base2[column_name].isnull()].Cargo.value_counts()[:20]

PRESIDENTE                 176707
VICEPRESIDENTE              70766
PRESIDENTA                  12928
PRESIDENT                    5619
VICEPRESIDENTA               4916
AUTOR                        2527
DORREGO GONZALEZ             2335
                             1916
MINISTRO DEL INTERIOR        1903
GARCIA ROYO                  1776
CERCOS PEREZ                 1439
MINISTRO DE JUSTICIA         1296
FUENTES NAVARRO              1144
EAJ-PNV                      1016
LABORDA MARTIN               1008
PRESIDENTE DEL GOBIERNO       959
MINISTRO DE FOMENTO           903
AUTORA                        880
SECRETARIO                    871
BASELGA GARCIA-ESCUDERO       850
Name: Cargo, dtype: int64

In [ ]:
from google.colab import files
base2.to_csv('base_parseada.csv', escapechar='\\')
files.download('base_parseada.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>